In [ ]:

#-------------  VERSION 1 ---------------#
#----   Prefered option since looks cleaner ----#

import pandas as pd     #   library to be able to access each of the csv files
import glob             #   library to be able to access the path
import numpy as np      #   library that sumplifies mathematical operations
import matplotlib.pyplot as plt
import joblib
import scipy as sp
from scipy.signal import filtfilt

path = "Gen/"
all_files = glob.glob(path + "*.csv")


li = []
for filename in all_files:
    flat_list = []
    emg = pd.read_csv(filename, index_col=1, delimiter=',', usecols=range(1,9))
    emg_correctmean = emg - np.mean(emg, axis=0)
    low_pass=20 # low: low-pass cut off frequency
    sfreq=1000 # sfreq: sampling frequency
    high_band=40
    low_band=450
    # emg: EMG data
    # high: high-pass cut off frequency
    
    # normalise cut-off frequencies to sampling frequency
    high_band = high_band/(sfreq/2)
    low_band = low_band/(sfreq/2)

    # create bandpass filter for EMG
    b1, a1 = sp.signal.butter(4, [high_band,low_band], btype='bandpass')

    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b1, a1, emg, axis=0)

    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)

    # create lowpass filter and apply to rectified signal to get EMG envelope
    low_pass = low_pass/(sfreq/2)
    b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
    emg_envelope = np.array(sp.signal.filtfilt(b2, a2, emg_rectified, axis=0))

    for sublist in emg_envelope:
        for item in sublist:
            flat_list.append(item)
    li.append(flat_list)

    print(li)

tar = ["Switch", "Freeze", "On/Off", "Forwards", "Backwards", "Left", "Right", "Up", "Down"]
n = 30
target = np.repeat(tar, n) # reperat each element n times
signal = np.array(li)